# Chapter 25: Composing types

In [1]:
newtype Identity a = Identity { runIdentity :: a }

In [2]:
:t id
:k Identity

id :: forall a. a -> a

Identity :: * -> *

In [3]:
newtype Compose f g a = Compose { getCompose :: f (g a) } deriving (Eq, Show)

In [4]:
:t (.)
:k Compose

(.) :: forall b c a. (b -> c) -> (a -> b) -> a -> c

Compose :: (* -> *) -> (* -> *) -> * -> *

In [5]:
instance Functor Identity where
    fmap f (Identity a) = Identity (f a)

In [6]:
instance (Functor f, Functor g) => Functor (Compose f g) where
    fmap f (Compose fga) = Compose $ (fmap . fmap) f fga

When you compose two Functors, you get another Functor - it means that Functors are *closed under composition*.

---

In [7]:
import Control.Applicative (liftA2)

instance (Applicative f, Applicative g) => Applicative (Compose f g) where
    pure = Compose . pure . pure
    (Compose f) <*> (Compose a) = Compose $ liftA2 (<*>) f a

When you compose two Applicatives, you get another Applicative - it means that Applicatives are *closed under composition*.

---

Turns out, it is not true for Monads though (generally).

In [27]:
import Control.Monad (join)

instance (Monad f, Monad g, Traversable g) => Monad (Compose f g) where
    (Compose fga) >>= a2cfgb = Compose fgb where
        a2fgb = getCompose . a2cfgb
        ga2gfgb = fmap a2fgb
        ga2fggb = sequenceA . ga2gfgb
        ga2fgb = fmap join . ga2fggb
        fgb = fga >>= ga2fgb        

But it works if first monad to compose is Traversable! Hmmm, I can conform to types, but will it actually be monad in terms of monad laws? Seems like it, check it out:

In [9]:
f :: a -> Compose [] Maybe a
f a = Compose [Just a, Nothing, Just a]

g :: b -> Compose [] Maybe b
g b = Compose [Nothing, Just b]

In [10]:
a = 10
m = Compose [Just a, Nothing]

In [11]:
return a >>= f
f a

Line 1: Monad law, left identity
Found:
return a >>= f
Why not:
f a

Compose {getCompose = [Just 10,Nothing,Just 10]}

Compose {getCompose = [Just 10,Nothing,Just 10]}

In [12]:
m >>= return
m

Line 1: Monad law, right identity
Found:
m >>= return
Why not:
m

Compose {getCompose = [Just 10,Nothing]}

Compose {getCompose = [Just 10,Nothing]}

In [13]:
(m >>= f) >>= g
m >>= (\x -> f x >>= g)

Line 2: Use >=>
Found:
\ x -> f x >>= g
Why not:
f Control.Monad.>=> g

Compose {getCompose = [Nothing,Just 10,Nothing,Nothing,Just 10,Nothing]}

Compose {getCompose = [Nothing,Just 10,Nothing,Nothing,Just 10,Nothing]}

---

In [14]:
instance (Foldable f, Foldable g) => Foldable (Compose f g) where
    foldMap am (Compose fga) = foldMap (foldMap am) fga

In [15]:
instance (Traversable f, Traversable g) => Traversable (Compose f g) where
    traverse a2hb (Compose fga) = Compose <$> traverse (traverse a2hb) fga

---

In [16]:
class Bifunctor p where
    {-# MINIMAL bimap | first, second #-}
    
    bimap :: (a -> b) -> (c -> d) -> p a c -> p b d
    bimap f g = first f . second g
    
    first :: (a -> b) -> p a c -> p b c
    first f = bimap f id
    
    second :: (b -> c) -> p a b -> p a c
    second = bimap id

In [17]:
data Deux a b = Deux a b

instance Bifunctor Deux where
    bimap a2b c2d (Deux a c) = Deux (a2b a) (c2d c)

In [18]:
newtype Const a b = Const a

instance Bifunctor Const where
    bimap a2b _ (Const a) = Const (a2b a)

In [19]:
data Drei a b c = Drei a b c

instance Bifunctor (Drei x) where
    bimap a2b c2d (Drei x a c) = Drei x (a2b a) (c2d c)

In [20]:
data SuperDrei a b c = SuperDrei a b

instance Bifunctor (SuperDrei x) where
    bimap a2b _ (SuperDrei x a) = SuperDrei x (a2b a)

In [21]:
newtype SemiDrei a b c = SemiDrei a

instance Bifunctor (SemiDrei x) where
    bimap _ _ (SemiDrei x) = SemiDrei x

In [22]:
data Quadriceps a b c d = Quadzzz a b c d

instance Bifunctor (Quadriceps x y) where
    bimap a2b c2d (Quadzzz x y a c) = Quadzzz x y (a2b a) (c2d c)

In [23]:
data Either a b = Left a | Right b

instance Bifunctor Either where
    bimap a2b _ (Left a) = Left (a2b a)
    bimap _ c2d (Right c) = Right (c2d c)

---

In [24]:
newtype IdentityT f a = IdentityT { runIdentityT :: f a } deriving (Eq, Show)

instance Functor Identity where
    fmap f (Identity a) = Identity (f a)
    
instance (Functor m) => Functor (IdentityT m) where
    fmap f (IdentityT fa) = IdentityT (fmap f fa)

In [25]:
instance Applicative Identity where
    pure = Identity
    
    (Identity f) <*> (Identity a) = Identity (f a)
    
instance (Applicative m) => Applicative (IdentityT m) where
    pure = IdentityT . pure
    
    (IdentityT fab) <*> (IdentityT fa) = IdentityT (fab <*> fa)

In [26]:
instance Monad Identity where
    (Identity a) >>= f = f a
    
instance Monad m => Monad (IdentityT m) where
    (IdentityT ma) >>= f = IdentityT $ ma >>= runIdentityT . f